In [22]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import spacy
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
nlp = spacy.load('en_core_web_sm')
import re
import string
from spacy.lang.en import English
import networkx as nx
import matplotlib.pyplot as plt
import nltk
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from spacy.matcher import Matcher 
from spacy.tokens import Span
from tqdm import tqdm
import bs4
import requests
from spacy import displacy

In [23]:
traindf = pd.read_csv(r"C:\Users\Hp\Desktop\train.csv")
testdf = pd.read_csv(r"C:\Users\Hp\Desktop\test.csv")
traindf

C:\Users\Hp\AppData\Local\Temp\ipykernel_15868\2467695041.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  testdf = pd.read_csv(r"C:\Users\Hp\Desktop\test.csv")


,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
...,...,...,...,...,...,...
404285,404285,433578,379845,How many keywords are there in the Racket prog...,How many keywords are there in PERL Programmin...,0
404286,404286,18840,155606,Do you believe there is life after death?,Is it true that there is life after death?,1
404287,404287,537928,537929,What is one coin?,What's this coin?,0
404288,404288,537930,537931,What is the approx annual cost of living while...,I am having little hairfall problem but I want...,0


# EDA

In [24]:
print("DataFrame Information:")
print("-" * 30)
print('DF Shape: ' ,traindf.shape)
print('number of Columns: ' ,len(traindf.columns))
print('number of Observations: ' ,len(traindf))
print('Number of values in traindf: ' , traindf.count().sum())
print('Total Number of Missing values in traindf: ' , traindf.isna().sum().sum())
print('percentage of Missing values : ' ,  "{:.2f}".format(traindf.isna().sum().sum()/traindf.count().sum() *100),'%')
print('Total Number of Duplicated records in traindf : ' , traindf.duplicated().sum().sum())
print('percentage of Duplicated values : ' ,  "{:.2f}".format(traindf.duplicated().sum().sum()/traindf.count().sum() *100),'%')

DataFrame Information:
------------------------------
DF Shape:  (404290, 6)
number of Columns:  6
number of Observations:  404290
Number of values in traindf:  2425737
Total Number of Missing values in traindf:  3
percentage of Missing values :  0.00 %
Total Number of Duplicated records in traindf :  0
percentage of Duplicated values :  0.00 %


In [25]:
print("DataFrame Information:")
print("-" * 30)
print('DF Shape: ' ,testdf.shape)
print('number of Columns: ' ,len(testdf.columns))
print('number of Observations: ' ,len(testdf))
print('Number of values in testdf: ' , testdf.count().sum())
print('Total Number of Missing values in testdf: ' , testdf.isna().sum().sum())
print('percentage of Missing values : ' ,  "{:.2f}".format(testdf.isna().sum().sum()/testdf.count().sum() *100),'%')
print('Total Number of Duplicated records in testdf : ' , testdf.duplicated().sum().sum())
print('percentage of Duplicated values : ' ,  "{:.2f}".format(testdf.duplicated().sum().sum()/testdf.count().sum() *100),'%')

DataFrame Information:
------------------------------
DF Shape:  (3563475, 3)
number of Columns:  3
number of Observations:  3563475
Number of values in testdf:  10690415
Total Number of Missing values in testdf:  10
percentage of Missing values :  0.00 %
Total Number of Duplicated records in testdf :  955535
percentage of Duplicated values :  8.94 %


In [26]:
traindf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [27]:
testdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3563475 entries, 0 to 3563474
Data columns (total 3 columns):
 #   Column     Dtype 
---  ------     ----- 
 0   test_id    object
 1   question1  object
 2   question2  object
dtypes: object(3)
memory usage: 81.6+ MB


In [28]:
traindf['is_duplicate'].value_counts()

0    255027
1    149263
Name: is_duplicate, dtype: int64

In [29]:
traindf.fillna('' , inplace= True)
testdf.fillna('' , inplace= True)
traindf.drop(['qid1' , 'qid2']  , axis= 1 , inplace= True)

In [30]:
true = traindf[traindf['is_duplicate'] == 1].head(1000)
false = traindf[traindf['is_duplicate'] == 0].head(1000)
traindf = pd.concat([true , false] , ignore_index=True )
testdf = testdf.head(1000)

In [31]:
traindf

,id,question1,question2,is_duplicate
0,5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1
1,7,How can I be a good geologist?,What should I do to be a great geologist?,1
2,11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?,1
3,12,What can make Physics easy to learn?,How can you make physics easy to learn?,1
4,13,What was your first sexual experience like?,What was your first sexual experience?,1
...,...,...,...,...
1995,1589,"When writing an official letter, if one does't...",How do I address a woman if I don't know if sh...,0
1996,1590,Is it OK to read two or three books at the sam...,What are the three must-read books for a first...,0
1997,1591,How do you put notes in a TI-84 Plus?,Do uncircumcised men urinate with a higher flo...,0
1998,1596,Why am I afraid to be myself?,Why am I afraid of telling my boyfriend I love...,0


# Preprocessing Functions

In [32]:
def get_entities(sent):
    ent1 = ""
    ent2 = ""

    prv_tok_dep = ""  # dependency tag of the previous token in the sentence
    prv_tok_text = ""  # the previous token in the sentence

    prefix = ""
    modifier = ""

    for tok in nlp(sent):
        # if the token is a punctuation mark, then move on to the next token
        if tok.dep_ != "punct":
            # check if the token is a compound word or not
            if tok.dep_ == "compound":
                prefix = tok.text
                # if the previous word was also a 'compound', then add the current word to it
                if prv_tok_dep == "compound":
                    prefix = prv_tok_text + " " + tok.text
            # check if the token is a modifier or not
            if tok.dep_.endswith("mod") == True:
                modifier = tok.text
                # if the previous word was also a 'compound', then add the current word to it
                if prv_tok_dep == "compound":
                    modifier = prv_tok_text + " " + tok.text
            # check if the token is a subject
            if tok.dep_.find("subj") == True:
                ent1 = modifier + " " + prefix + " " + tok.text
                prefix = ""
                modifier = ""
                prv_tok_dep = ""
                prv_tok_text = ""
            # check if the token is an object
            if tok.dep_.find("obj") == True:
                ent2 = modifier + " " + prefix + " " + tok.text
            # update variables
            prv_tok_dep = tok.dep_
            prv_tok_text = tok.text

    return [ent1.strip(), ent2.strip()]


In [33]:
def clean_text(x):
    text = re.sub('(\d+)','',x)   
    text = text.lower()
    return text
def remove_punct(x):
    text_without_puct = [t for t in x if t not in string.punctuation]
    text_without_puct = ''.join(text_without_puct)
    return text_without_puct
stemm = PorterStemmer()
def stemmer(x):
    tokens = word_tokenize(x)    
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(token) for token in tokens]
    preprocessed_text = ' '.join(tokens)

    return preprocessed_text    

In [34]:
def get_relation(sent):
    doc = nlp(sent)

    # Matcher class object
    matcher = Matcher(nlp.vocab)

    # Define the pattern
    pattern = [{'DEP': 'ROOT'},
               {'DEP': 'prep', 'OP': "?"},
               {'DEP': 'agent', 'OP': "?"},
               {'POS': 'ADJ', 'OP': "?"}]

    # Wrap the pattern in a list when adding to the Matcher
    matcher.add("matching_1", [pattern])

    matches = matcher(doc)
    k = len(matches) - 1

    span = doc[matches[k][1]:matches[k][2]]

    return span.text

# Cleaning texts

In [35]:
traindf['question2'] = traindf['question2'].apply(clean_text)
traindf['question2'] = traindf['question2'].apply(remove_punct)
traindf['question2'] = traindf['question2'].apply(stemmer)

traindf['question1'] = traindf['question1'].apply(clean_text)
traindf['question1'] = traindf['question1'].apply(remove_punct)
traindf['question1'] = traindf['question1'].apply(stemmer)


traindf['question1'] = traindf['question1'].apply(clean_text)
traindf['question1'] = traindf['question1'].apply(remove_punct)
traindf['question1'] = traindf['question1'].apply(stemmer)


traindf['question2'] = traindf['question2'].apply(clean_text)
traindf['question2'] = traindf['question2'].apply(remove_punct)
traindf['question2'] = traindf['question2'].apply(stemmer)


# Finding Entity pairs and relations

In [36]:
entity_pairs_q1 = []
for i in tqdm(traindf["question1"]):
    entity_pairs_q1.append(get_entities(i))
relations_q1 = [get_relation(i) for i in tqdm(traindf['question1'])]


entity_pairs_q2 = []
for i in tqdm(traindf["question2"]):
    entity_pairs_q2.append(get_entities(i))
relations_q2 = [get_relation(i) for i in tqdm(traindf['question2'])]


100%|██████████| 2000/2000 [00:11<00:00, 172.61it/s]


In [37]:
additional_df = pd.DataFrame(entity_pairs_q1, columns=['SubjectQ1' , 'ObjectQ1'])
additional_df['relationsQ1'] = relations_q1

additional_df2 = pd.DataFrame(entity_pairs_q2, columns=['SubjectQ2' , 'ObjectQ2'])
additional_df2['relationsQ2'] = relations_q2

traindf = pd.concat([traindf, additional_df, additional_df2], axis=1)

traindf

,id,question1,question2,is_duplicate,SubjectQ1,ObjectQ1,relationsQ1,SubjectQ2,ObjectQ2,relationsQ2
0,5,astrolog i am a capricorn sun cap moon and cap...,im a tripl capricorn sun moon and ascend in ca...,1,capricorn risingwhat that,me,am,i,capricorn sun me,m
1,7,how can i be a good geologist,what should i do to be a great geologist,1,how i,,be,i,what,do
2,11,how do i read and find my youtub comment,how can i see all my youtub comment,1,how i,youtub comment,read,how i,youtub comment,see
3,12,what can make physic easi to learn,how can you make physic easi to learn,1,physic,,make physic,physic easi,,make physic
4,13,what wa your first sexual experi like,what wa your first sexual experi,1,sexual experi,,wa,sexual experi,,wa
...,...,...,...,...,...,...,...,...,...,...
1995,1589,when write an offici letter if one doest know ...,how do i address a woman if i dont know if she...,0,woman,one her,write,she,woman,address
1996,1590,is it ok to read two or three book at the same...,what are the three mustread book for a firstti...,0,it,one book,is,three mustread book,startup founder,are
1997,1591,how do you put note in a ti plu,do uncircumci men urin with a higher flow rate,0,how you,ti plu,put,,higher flow rate,uncircumci
1998,1596,whi am i afraid to be myself,whi am i afraid of tell my boyfriend i love him,0,i,,am,i,him,am


In [38]:
entity_pairs_q1 = []
for i in tqdm(testdf["question1"]):
    entity_pairs_q1.append(get_entities(i))
relations_q1 = [get_relation(i) for i in tqdm(testdf['question1'])]


entity_pairs_q2 = []
for i in tqdm(testdf["question2"]):
    entity_pairs_q2.append(get_entities(i))
relations_q2 = [get_relation(i) for i in tqdm(testdf['question2'])]


100%|██████████| 1000/1000 [00:05<00:00, 171.11it/s]


In [39]:
additional_df = pd.DataFrame(entity_pairs_q1, columns=['SubjectQ1' , 'ObjectQ1'])
additional_df['relationsQ1'] = relations_q1

additional_df2 = pd.DataFrame(entity_pairs_q2, columns=['SubjectQ2' , 'ObjectQ2'])
additional_df2['relationsQ2'] = relations_q2

testdf = pd.concat([testdf, additional_df, additional_df2], axis=1)

testdf

,test_id,question1,question2,SubjectQ1,ObjectQ1,relationsQ1,SubjectQ2,ObjectQ2,relationsQ2
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,How Surface Pro,iPad Pro,compare with,Why Microsoft,core m3,choose
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,How hair it,How hair much,cost,,much cost,require
2,2,What but is the best way to send money from Ch...,What you send money to China?,,best US,is,you,China,send
3,3,Which food not emulsifiers?,What foods fibre?,,,emulsifiers,,foods,fibre
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,,,start,How I,How their,start
...,...,...,...,...,...,...,...,...,...
995,995,How is RAU's IAS study circle (Bangalore branc...,How Rau's IAS Study Circle in the Bangalore br...,UPSC coaching,Bangalore itself,is,,How IAS preparations,Circle in
996,996,"How NFL, can you try to kick a field goal on a...",What is the point of having good on the NFL go...,you,other 4th,happens,point,NFL goal lines,is
997,997,How do is campus life at St. Thomas College of...,How is campus life layer placements at St Thom...,,How St. Thomas Engineering,is,How life layer placements,St Thomas Engineering,is
998,998,How do I earn money with prefer?,What is on YouTube?,How I,prefer,earn,What,YouTube,is on


In [40]:
testdf.columns

Index(['test_id', 'question1', 'question2', 'SubjectQ1', 'ObjectQ1',
       'relationsQ1', 'SubjectQ2', 'ObjectQ2', 'relationsQ2'],
      dtype='object')

# Creating Embeddings

In [41]:
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np
y = traindf.pop('is_duplicate')

ft = ['question1', 'question2', 'SubjectQ1', 'ObjectQ1',
       'relationsQ1', 'SubjectQ2', 'ObjectQ2', 'relationsQ2']

sentences = []
for i in ft:
    s = traindf['question1'].apply(str.split).tolist()
    sentences = sentences + s

word2vec_model = Word2Vec(sentences, min_count=1)

xtrain, xtest, ytrain, ytest = train_test_split(traindf, y, test_size=0.2, random_state=42)


def question_to_vector(question, model):
    words = question.split()
    vectors = [model.wv[word] for word in words if word in model.wv]
    if vectors:
        return np.mean(vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

L = []
for i in ft:
    L.append(np.array([question_to_vector(q, word2vec_model) for q in xtrain[i]]))

xtrain_combined = np.concatenate((L), axis=1)


L = []
for i in ft:
    L.append(np.array([question_to_vector(q, word2vec_model) for q in xtest[i]]))

xtest_combined = np.concatenate((L), axis=1)


# Modelling


In [43]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

logistic_regression = LogisticRegression()

param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000], 'max_iter': [100, 200, 300, 400, 500]}

grid_search = GridSearchCV(logistic_regression, param_grid, cv=5, scoring='accuracy')

grid_search.fit(xtrain_combined, ytrain)

print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

best_model = grid_search.best_estimator_
predictions = best_model.predict(xtest_combined)

accuracy = accuracy_score(ytest, predictions)
print("Test Set Accuracy:", accuracy)


c:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

Best Parameters: {'C': 10, 'max_iter': 300}
Best Accuracy: 0.6137500000000001
Test Set Accuracy: 0.595


c:\Users\Hp\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create Random Forest classifier
random_forest = RandomForestClassifier()

# Define parameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# Create grid search object
grid_search = GridSearchCV(random_forest, param_grid, cv=5, scoring='accuracy')

# Fit grid search to the data
grid_search.fit(xtrain_combined, ytrain)

# Print the best parameters and corresponding accuracy
print("Best Parameters:", grid_search.best_params_)
print("Best Accuracy:", grid_search.best_score_)

# Use the best model for predictions
best_model = grid_search.best_estimator_
predictions = best_model.predict(xtest_combined)

# Evaluate the accuracy on the test set
accuracy = accuracy_score(ytest, predictions)
print("Test Set Accuracy:", accuracy)


Best Parameters: {'max_depth': 10, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Accuracy: 0.656875
Test Set Accuracy: 0.6075


In [46]:
L = []
for i in ft:
    L.append(np.array([question_to_vector(q, word2vec_model) for q in testdf[i]]))
    
testdf_combined = np.concatenate((L), axis=1)
preds = best_model.predict(testdf_combined)

d = {
    'test_id' : testdf['test_id'],	
    'is_duplicate' : preds
}

sub = pd.DataFrame(d)

import os
path = "C:\\Users\\Hp\\Desktop\\"
sub.to_csv(os.path.join(path, 'nlp_submission.csv'))

- I had to slice the dataset due to lack of computational power i couldnt process the whole setas it was too large
- I think we can improve the accuracy if we could train on the all 40K training records